In [45]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Data getting, cleaning, and exploring
from acquire import get_connection, new_telco_data, get_telco_data
from prepare import clean_telco, impute_mode, train_validate_test_split,impute_mode, prep_telco_data
import explore

# Python without these is hard
import pandas as pd
import numpy as np
from pydataset import data

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import graphviz
from graphviz import Graph

# Data Aquisition 

In [ ]:
sql_query = '''
    SELECT *
    FROM customers
    JOIN contract_types USING(`contract_type_id`)
    JOIN internet_service_types USING(`internet_service_type_id`)
    JOIN payment_types USING(payment_type_id);
            '''

In [ ]:
get_db_url = f'mysql+pymysql://{user}:{password}@{host}/telco_churn'

In [ ]:
df = pd.read_sql(sql_query, get_db_url)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe().T

# Data Preparation

In [ ]:
df.replace('No phone service', 'No', inplace=True)
df.replace('No internet service', 'No', inplace=True)
  

In [ ]:
dummy_df = pd.get_dummies(df[['gender']], drop_first=True)
dummy2_df = pd.get_dummies(df[['contract_type','internet_service_type','payment_type']], drop_first=False)
dropcols = [
            'gender',
            'payment_type_id',
            'internet_service_type_id',
            'contract_type_id',
            'customer_id',
            'contract_type',
            'internet_service_type',
            'payment_type',
            'internet_service_type_none'
               ]

In [ ]:
df = pd.concat([df, dummy_df,dummy2_df], axis=1)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('(','').str.replace(')','')

In [ ]:
df.drop(columns=dropcols, inplace=True)

In [ ]:
df.columns = [
    'senior_citizen','partner','dependents','tenure','phone_service','multiple_lines',
    'online_security','online_backup','device_protection','tech_support','streaming_tv','streaming_movies',
    'paperless_billing','monthly_charges','total_charges','churn','gender_male','month-to-month_contract', 
    'one_year_contract','two_year_contract', 'dsl_internet','fiber_optic_internet','payment_type_bank_transfer',
    'payment_type_credit_card', 'payment_type_e_check','payment_type_mailed_check'
            ]

In [ ]:
df.total_charges = pd.to_numeric(df.total_charges, errors='coerce').astype('float64')

In [65]:
quant_vars = ['tenure','monthly_charges','total_charges']
cat_vars = list((df.columns).drop(quant_vars))

In [33]:
df1 = df[df.isna().any(axis=1)].drop(columns=cat_vars)

In [36]:
df1.head()

,tenure,monthly_charges,total_charges
1878,0,56.05,NaN
1949,0,61.90,NaN
2029,0,73.35,NaN
2048,0,52.55,NaN
2132,0,80.85,NaN


In [37]:
df1.total_charges = df1.total_charges.fillna(value=0)

In [38]:
df1.total_charges.isnull().sum()

0

In [43]:
df = df.replace("Yes", 1).replace("No", 0)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   senior_citizen              7043 non-null   int64  
 1   partner                     7043 non-null   int64  
 2   dependents                  7043 non-null   int64  
 3   tenure                      7043 non-null   int64  
 4   phone_service               7043 non-null   int64  
 5   multiple_lines              7043 non-null   int64  
 6   online_security             7043 non-null   int64  
 7   online_backup               7043 non-null   int64  
 8   device_protection           7043 non-null   int64  
 9   tech_support                7043 non-null   int64  
 10  streaming_tv                7043 non-null   int64  
 11  streaming_movies            7043 non-null   int64  
 12  paperless_billing           7043 non-null   int64  
 13  monthly_charges             7043 

In [59]:
train, validate, test = train_validate_test_split(df)
target = "df.churn"

In [60]:
train.shape, validate.shape, test.shape

((3943, 26), (1691, 26), (1409, 26))

In [63]:
explore.explore_bivariate_quant(train, target, quant_var)

NameError: name 'quant_var' is not defined